<a href="https://colab.research.google.com/github/EjbejaranosAI/ALIA/blob/main/ALIA_implementation_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ALIA Setup

In [1]:
import sys
import os
import tensorflow as tf


colab = 'google.colab' in sys.modules
if colab:
  # noinspection PyBroadException
  try:
    !git clone https://github.com/lisadunlap/ALIA.git
  except Exception:
    pass

if colab:
  os.environ['PROJECT_DIR'] = project_dir = '/content/ALIA'
  replace = True
  if os.path.isdir(project_dir):
    choice = input("Project already exists in folder "+
          "{}\nDelete the files and pull again? Enter Y/(N):\n"
          .format(project_dir))
    if choice.lower()=='y':
      !rm -r $PROJECT_DIR
      print("Deleted folder {}".format(project_dir))
    else:
      replace = False
      print("Nothing was changed.")
  if replace:
    !cd /content && git clone https://github.com/lisadunlap/ALIA.git
    print("Project cloned to " + project_dir)
  os.chdir(project_dir)
  print("Done.")

Cloning into 'ALIA'...
remote: Enumerating objects: 305, done.
remote: Counting objects: 100% (305/305), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 305 (delta 167), reused 237 (delta 114), pack-reused 0
Receiving objects: 100% (305/305), 5.83 MiB | 12.21 MiB/s, done.
Resolving deltas: 100% (167/167), done.
Project already exists in folder /content/ALIA
Delete the files and pull again? Enter Y/(N):
n
Nothing was changed.
Done.


In [2]:
!pip install -q numpy clip matplotlib regex wilds omegaconf wandb scikit-image torch torchfile torchtext torchvision tqdm transformers attrdict diffusers[torch] tyro git+https://github.com/ildoonet/cutmix git+https://github.com/openai/CLIP.git cleanlab

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ...

In [27]:
#!sudo apt install tree
!tree

.
├── caption.py
├── captions
│   ├── Cub2011.csv
│   ├── iWildCamMini.csv
│   └── Planes.csv
├── configs
│   ├── base.yaml
│   ├── Cub2011
│   │   ├── alia.yaml
│   │   ├── base_mlp.yaml
│   │   ├── base.yaml
│   │   ├── extra.yaml
│   │   └── txt2img.yaml
│   ├── iWildCamMini
│   │   ├── alia_instruct.yaml
│   │   ├── alia.yaml
│   │   ├── base.yaml
│   │   ├── extra.yaml
│   │   └── txt2img.yaml
│   └── Planes
│       ├── alia.yaml
│       ├── base.yaml
│       ├── extra.yaml
│       ├── img2img.yaml
│       ├── instruct_generic.yaml
│       ├── instruct.yaml
│       └── txt2img.yaml
├── data
│   ├── iwildcam_v2.0
│   │   ├── test_subset.csv
│   │   ├── train_extra_subset.csv
│   │   ├── train_subset.csv
│   │   └── val_subset.csv
│   └── planes_ext.csv
├── datasets
│   ├── base.py
│   ├── cub.py
│   ├── __init__.py
│   ├── planes.py
│   ├── __pycache__
│   │   ├── base.cpython-310.pyc
│   │   ├── cub.cpython-310.pyc
│   │   ├── __init__.cpython-310.pyc
│   │   ├── planes.cpython-31

# ALIA implementation
#### API KEY WANBS: ************

## Prompt generation

In [31]:
import urllib.request

url = "https://data.caltech.edu/records/65de6-vp158/files/CUB_200_2011.tgz?download=1"
filename = "/content/dataset/CUB_200_2011.tgz"

urllib.request.urlretrieve(url, filename)



('/content/CUB_200_2011.tgz', <http.client.HTTPMessage at 0x7f78235748b0>)

In [35]:
!tar -xzf /content/dataset/CUB_200_2011.tgz  -C "/content/dataset/" && rm /content/dataset/CUB_200_2011.tgz

In [36]:
!python caption.py --config configs/Cub2011/base.yaml

2023-08-17 20:42:35.718290: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
wandb: Currently logged in as: ejbejaranos22. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.8
wandb: Run data is saved locally in /content/ALIA/wandb/run-20230817_204240-lpwz0srv
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run deep-violet-8
wandb: ⭐️ View project at https://wandb.ai/ejbejaranos22/ALIA-Cub2011
wandb: 🚀 View run at https://wandb.ai/ejbejaranos22/ALIA-Cub2011/runs/lpwz0srv
==> Preparing data..
/shared/lisabdunlap/data/CUB-200-Painting/classes.txt
Traceback (most recent call last):
  File "/content/ALIA/caption.py", line 63, in <module>
    trainset, valset, testset = get_filtered_dataset(args, None, None)
  File "/content/ALIA/helpers/load_dataset.py", line 149, in get_filtered_dataset
    trainset, valset, testset, extraset = get_dataset(args.data.base_dataset, transform, val_transform, r

## Creating images from prompts

- Generating images about arachnophobia

In [3]:
!python editing_methods/txt2img_example.py --prompt "Arachnophobia" --n 20

2023-08-17 10:37:35.733474: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 2
wandb: You chose 'Use an existing W&B account'
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Tracking run with wandb version 0.15.8
wandb: Run data is saved locally in /content/ALIA/wandb/run-20230817_103753-8jes68c5
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run Arachnophobia
wandb: ⭐️ View project at https://wandb.ai/ejbejaranos22/Text-2-Image
wandb: 🚀 View run at https://wandb.ai/ejbejaranos22/Text-2-Image/runs/

- Generating images about "Cats in winter"

In [4]:
!python editing_methods/txt2img_example.py --prompt "Cats in winter" --n 20

2023-08-17 10:48:52.441210: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
wandb: Currently logged in as: ejbejaranos22. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.8
wandb: Run data is saved locally in /content/ALIA/wandb/run-20230817_104856-o5dulnah
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run Cats in winter
wandb: ⭐️ View project at https://wandb.ai/ejbejaranos22/Text-2-Image
wandb: 🚀 View run at https://wandb.ai/ejbejaranos22/Text-2-Image/runs/o5dulnah
Loading pipeline components...: 100% 6/6 [00:19<00:00,  3.19s/it]
Generating images for prompt: Cats in winter
  0% 0/10 [00:00<?, ?it/s]
  0% 0/50 [00:00<?, ?it/s]
  2% 1/50 [00:03<03:15,  3.98s/it]
  4% 2/50 [00:04<01:25,  1.79s/it]
  6% 3/50 [00:04<00:51,  1.09s/it]
  8% 4/50 [00:04<00:35,  1.31it/s]
 10% 5/50 [00:05<00:26,  1.72it/s]
 12% 6/50 [00:05<00:20,  2.13it/s]
 14% 7/50 [00:05<00:17,  2.50it/s]
 16% 8/50 [00

In [14]:
!python editing_methods/txt2img_example.py --prompt "street images in winter snowing but with sun, some traffic are observed and 2 dogs with a guy walknig " --n 20

2023-08-17 19:28:25.221889: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
wandb: Currently logged in as: ejbejaranos22. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.8
wandb: Run data is saved locally in /content/ALIA/wandb/run-20230817_192830-3veicss3
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run street images in winter snowing but with sun, some traffic are observed and 2 dogs with a guy walknig 
wandb: ⭐️ View project at https://wandb.ai/ejbejaranos22/Text-2-Image
wandb: 🚀 View run at https://wandb.ai/ejbejaranos22/Text-2-Image/runs/3veicss3
Loading pipeline components...: 100% 6/6 [00:03<00:00,  2.00it/s]
Generating images for prompt: street images in winter snowing but with sun, some traffic are observed and 2 dogs with a guy walknig 
  0% 0/10 [00:00<?, ?it/s]
  0% 0/50 [00:00<?, ?it/s]
  2% 1/50 [00:04<03:38,  4.47s/it]
  4% 2/50 [00:04<01:34,  1.98s/it]
  6% 3/50 [0

## Creating images from images and prompts

- ADD THIS TO img2img.py:
```python
import sys
print('before')
print(sys.path)
sys.path.append('/content/ALIA')
print('After')
print(sys.path)
```


- ADD THIS TO img2img_example.py:
```python
    wandb.init(project="Image-2-Image", name=f"{prompt_str}-{args.strength}-{args.guidance}", group=args.im_path, config=args, entity='ejbejaranos22')

```

In [41]:
!python editing_methods/img2img_example.py --prompt "Snowing with rain" --n 20

2023-08-17 21:01:38.034886: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
wandb: Currently logged in as: ejbejaranos22. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.8
wandb: Run data is saved locally in /content/ALIA/wandb/run-20230817_210141-h7lom9ry
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run Snowing_with_rain-0.6-5
wandb: ⭐️ View project at https://wandb.ai/ejbejaranos22/Image-2-Image
wandb: 🚀 View run at https://wandb.ai/ejbejaranos22/Image-2-Image/runs/h7lom9ry
Loading pipeline components...: 100% 6/6 [00:09<00:00,  1.62s/it]
Editing image ./ex_imgs/giraffe.png with prompt: Snowing with rain
100% 30/30 [00:06<00:00,  4.67it/s]
100% 30/30 [00:06<00:00,  4.77it/s]
100% 30/30 [00:06<00:00,  4.72it/s]
100% 30/30 [00:06<00:00,  4.68it/s]
100% 30/30 [00:06<00:00,  4.63it/s]
100% 30/30 [00:06<00:00,  4.62it/s]
100% 30/30 [00:06<00:00,  4.57it/s]
100% 30/30 [00:06<00:00,  4

In [ ]:
!python editing_methods/img2img_example.py --prompt "Snow in the street and sunny in the sky" --n 20

2023-08-17 21:09:25.478683: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
wandb: Currently logged in as: ejbejaranos22. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.8
wandb: Run data is saved locally in /content/ALIA/wandb/run-20230817_210930-pul1920o
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run Snow_in_the_street_and_sunny_in_the_sky-0.6-5
wandb: ⭐️ View project at https://wandb.ai/ejbejaranos22/Image-2-Image
wandb: 🚀 View run at https://wandb.ai/ejbejaranos22/Image-2-Image/runs/pul1920o
Loading pipeline components...: 100% 6/6 [00:09<00:00,  1.62s/it]
Editing image ./ex_imgs/giraffe.png with prompt: Snow in the street and sunny in the sky
100% 30/30 [00:06<00:00,  4.67it/s]
100% 30/30 [00:06<00:00,  4.71it/s]
100% 30/30 [00:06<00:00,  4.68it/s]
100% 30/30 [00:06<00:00,  4.65it/s]
100% 30/30 [00:06<00:00,  4.59it/s]
100% 30/30 [00:06<00:00,  4.55it/s]
100% 30/30 [00:06<0

In [38]:
!python editing_methods/img2img.py --dataset Cub2011 --prompt "a photo of a {} bird on rocks." --n 2

before
['/content/ALIA/editing_methods', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages']
After
['/content/ALIA/editing_methods', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/content/ALIA']
2023-08-17 20:55:18.118650: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
wandb: Currently logged in as: ejbejaranos22. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.8
wandb: Run data is saved locally in /content/ALIA/wandb/run-20230817_205524-lgiw0bgi
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run Cub2011-a photo of a {} bird on rocks.-0.6-5
wandb: ⭐️ View project at https://wandb.ai/ejbejaranos22/Image-2-Image
wandb: 🚀 View run at 

In [22]:
!python editing_methods/img2img_example.py

2023-08-17 20:10:10.107270: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [24]:
!python filtering/filter.py --config configs/Cub2011/alia.yaml filter.load=false

before
['/content/ALIA/filtering', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages']
After
['/content/ALIA/filtering', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/content/ALIA']
wandb: Currently logged in as: ejbejaranos22. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.8
wandb: Run data is saved locally in /content/ALIA/wandb/run-20230817_201310-3lskj4ix
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run autumn-snowflake-1
wandb: ⭐️ View project at https://wandb.ai/ejbejaranos22/ALIA-filter
wandb: 🚀 View run at https://wandb.ai/ejbejaranos22/ALIA-filter/runs/3lskj4ix
100%|███████████████████████████████████████| 890M/890M [00:10<00:00, 89.1MiB/s]
==> Preparing data..
Roots

.
├── caption.py
├── captions
│   ├── Cub2011.csv
│   ├── iWildCamMini.csv
│   └── Planes.csv
├── configs
│   ├── base.yaml
│   ├── Cub2011
│   │   ├── alia.yaml
│   │   ├── base_mlp.yaml
│   │   ├── base.yaml
│   │   ├── extra.yaml
│   │   └── txt2img.yaml
│   ├── iWildCamMini
│   │   ├── alia_instruct.yaml
│   │   ├── alia.yaml
│   │   ├── base.yaml
│   │   ├── extra.yaml
│   │   └── txt2img.yaml
│   └── Planes
│       ├── alia.yaml
│       ├── base.yaml
│       ├── extra.yaml
│       ├── img2img.yaml
│       ├── instruct_generic.yaml
│       ├── instruct.yaml
│       └── txt2img.yaml
├── data
│   ├── iwildcam_v2.0
│   │   ├── test_subset.csv
│   │   ├── train_extra_subset.csv
│   │   ├── train_subset.csv
│   │   └── val_subset.csv
│   └── planes_ext.csv
├── datasets
│   ├── base.py
│   ├── cub.py
│   ├── __init__.py
│   ├── planes.py
│   ├── __pycache__
│   │   ├── base.cpython-310.pyc
│   │   ├── cub.cpython-310.pyc
│   │   ├── __init__.cpython-310.pyc
│   │   ├── planes.cpython-31